In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import numpy as np
from torch.utils.data import DataLoader

In [2]:
test_df = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Disaster_Tweets_Classification/Data/test.csv')

In [3]:
train_df = pd.read_csv('/Users/merlesteffen/Documents/GitHub/Disaster_Tweets_Classification/Data/train.csv')

# Split Dataset Test/Train

In [4]:
df_train, df_test = train_test_split(train_df, test_size=0.2, random_state=42)

# BERT

## Tokenization

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# Tokenize the text
train_encodings = tokenizer(df_train['text'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(df_test['text'].tolist(), truncation=True, padding=True, max_length=512)

## Creating Datasets

In [7]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
# Create datasets
train_dataset = TextDataset(train_encodings, df_train['target'].tolist())
test_dataset = TextDataset(test_encodings, df_test['target'].tolist())

## Fine-Tuning BERT

In [9]:
# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
)

In [11]:
# Create a Trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
)

/Users/merlesteffen/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/merlesteffen/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [12]:
# Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=2286, training_loss=0.39576279981451296, metrics={'train_runtime': 472.9555, 'train_samples_per_second': 38.629, 'train_steps_per_second': 4.833, 'total_flos': 788654832890400.0, 'train_loss': 0.39576279981451296, 'epoch': 3.0})

# Make Predictions

In [13]:
# Make predictions
predictions = trainer.predict(test_dataset)

# Convert predictions to labels
pred_labels = np.argmax(predictions.predictions, axis=1)

# Evaluate Model

In [14]:
# Calculate accuracy
accuracy = accuracy_score(df_test['target'], pred_labels)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8200919238345371


# Train on entire Dataset (Train)

## Tokenization

In [6]:
# Tokenize the training text
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, max_length=512)

# Tokenize the test text
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True, max_length=512)

## Convert to Tensors and Create Datasets

In [7]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [10]:
# Create datasets
train_data = TextDataset(train_encodings, train_df['target'].tolist())
test_data = TextDataset(test_encodings)

## Create DataLoader

In [11]:
# Define batch size
batch_size = 8

# Create DataLoaders
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


## Train the model

In [12]:
# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    logging_dir='./logs',
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/merlesteffen/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/merlesteffen/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [13]:
# Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=2856, training_loss=0.3902771573106782, metrics={'train_runtime': 587.1036, 'train_samples_per_second': 38.901, 'train_steps_per_second': 4.865, 'total_flos': 985883291099280.0, 'train_loss': 0.3902771573106782, 'epoch': 3.0})

## Make Predictions

In [14]:
# Make predictions
predictions = trainer.predict(test_data)

# Convert predictions to labels
pred_labels = np.argmax(predictions.predictions, axis=1)

# Assuming you want to attach these predictions to your test_df
test_df['predicted_label'] = pred_labels

# Create Submission File

In [15]:
# Create the submission DataFrame
submission_df = test_df[['id', 'predicted_label']]

# Rename 'predicted_label' to 'target'
submission_df = submission_df.rename(columns={'predicted_label': 'target'})

# Save to CSV file
submission_df.to_csv('/Users/merlesteffen/Documents/GitHub/Disaster_Tweets_Classification/Data/submissions/submissions.csv', index=False)

In [18]:
submission_df

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [23]:
submission_df

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
